Este colab es únicamente utilizado para crear los grafos a partir de los datos de FoF.

Hola, besos <3.

In [ ]:
import os
import time
import numpy as np
import networkx as nx
import igraph as ig
from scipy.spatial import Delaunay
from typing import Literal
import readfof  # Este modulo es de Pylians


fof_root: str = f"./quijote/FoF"
graphs_root: str = f"./quijote/grafos"
snapnums: list = [0, 1, 2, 3, 4]
z_dict: dict[int, float] = {4: 0.0, 3: 0.5, 2: 1.0, 1: 2.0, 0: 3.0}


def distance(point1: list, point2: list) -> float:
    """ Calcula la distancia entre dos puntos de la misma dimensión """
    assert len(point1) == len(point2), "El tamaño debe ser el mismo, gilipollas..."
    return np.sqrt(sum((p1 - p2)**2 for p1, p2 in zip(point1, point2)))


def tolist(G: nx.Graph, node, prefix: Literal['', 'v'] = '') -> list:
    """ En el grafo G (son los que se están guardando como archivos) hay atributos que
    son originalmente vectores (caso de la velocidad y la posición de los halos), y
    para esos casos, los atributos para cada uno de los nodos fue representado de la
    siguiente manera:

    Para el caso de la posición, los atributos se descomponieron en `x`, `y` y `z`.
    Para el caso de la velocidad, los atributos se descomponieron en `vx`, `vy` y `vz`.

    Entonces, lo que realiza la función es que dado un nodo (`node`) de uno de esos grafos (`G`)
    que estamos guardando en archivos, crea su respectiva lista de las posiciones (`prefix=''`) o
    las velocidades (`prefix='v`).

    Hasta el momento, `prefix` únicamente puede tomar esos valores, puesto que esos son los únicos
    que son guardados en listas """
    return [G.nodes[node][f'{prefix}{i}'] for i in ('x', 'y', 'z')]


assert os.path.exists(fof_root)
assert os.path.exists(graphs_root)

## Guardar Grafo

In [ ]:
def save_graph(FoF, snapnum, save_path) -> None:
    redshift = z_dict[snapnum]

    pos_h = FoF.GroupPos / 1e3               # Posición de los halos en Mpc/h
    mass  = FoF.GroupMass * 1e10             # Masa de los halos en Msun/h
    vel_h = FoF.GroupVel * (1.0 + redshift)  # Velocidad particulas de los halos km/s
    Npart = FoF.GroupLen                     # Número de partículas CDM en los halos

    t0 = time.perf_counter()
    delaunay3d = Delaunay(pos_h)
    simplices3d = delaunay3d.simplices
    t1 = time.perf_counter()-t0

    G = nx.Graph()

    t0 = time.perf_counter()
    for i, point in enumerate(pos_h):
        pos, vel = point, vel_h[i]

        G.add_node(i, x=pos[0], y=pos[1], z=pos[2],
                vx=vel[0], vy=vel[1], vz=vel[2],
                mass=mass[i], npart=Npart[i])
    t2 = time.perf_counter()-t0

    t0 = time.perf_counter()
    for path in simplices3d.tolist():
        path.append(path[0])
        nx.add_path(G, path)
    t3 = time.perf_counter()-t0

    t0 = time.perf_counter()
    distances = {edge: distance(tolist(G, edge[0]), tolist(G, edge[1])) for edge in G.edges}
    nx.set_edge_attributes(G, values=distances, name='distance')
    t4 = time.perf_counter()-t0

    print(f'Tiempo total: {t1+t2+t3+t4} s')

    nx.write_graphml(G, save_path)


for simu in os.listdir(fof_root):
    # Ruta del archivo de la simulaciones
    fof_tmp_path = f'{fof_root}/{simu}'

    # Crear el respectivo para el grafo en caso de no estar
    graph_tmp_path = f'{graphs_root}/{simu}'
    if not os.path.exists(graph_tmp_path):
        os.mkdir(graph_tmp_path)

    # Iterar sobre las muestras de cada simulación
    for i in os.listdir(fof_tmp_path):
        snapdir = f'{fof_tmp_path}/{i}'

        graph_tmp_path_i = f'{graph_tmp_path}/{i}'
        if not os.path.exists(graph_tmp_path_i):
            os.mkdir(graph_tmp_path_i)

        # Iterar sobre todos los snapnums descargados en la carpeta de FoF
        for snapnum in snapnums:
            # Si ya existe el archivo, se lo salta. Para evitar trabajo doble
            graph_path = f'{graph_tmp_path_i}/{simu}_{snapnum:03}.graphml'
            if os.path.exists(graph_path):
                continue

            # Crea los grafos solo en los archivos que no tienen uno asignado
            try:
                FoF = readfof.FoF_catalog(snapdir, snapnum, read_IDs=False)
            except FileNotFoundError:
                continue
            print(f'{graph_path}  ---  ', end='')
            save_graph(FoF, snapnum, graph_path)
    print()

## Guardar grafos aleatorios

In [ ]:
def generate_random_points(n: int, value_min: float, value_max: float) -> np.ndarray:
    """
    Genera un arreglo de n puntos tridimensionales aleatorios

    Args:
        n (int): Cantidad de puntos

    Returns:
        numpy.ndarray: Arreglo de n puntos tridimensionales.
    """
    return np.random.uniform(value_min, value_max, (n, 3))

In [ ]:
# Cantidad máxima de nodos por cada snapnum
max_nodes_snapnums = np.array([5252, 46546, 201193, 315211, 410006])

for snapnum, n in enumerate(max_nodes_snapnums):
    graph_path = f'{graphs_root}/random_{snapnum:03}.graphml'
    pos = generate_random_points(int(n * 1.5), 0, 1000)
    print(f'{graph_path}  ---  ', end='')

    t0 = time.perf_counter()
    delaunay3d = Delaunay(pos)
    simplices3d = delaunay3d.simplices
    t1 = time.perf_counter()-t0

    G = nx.Graph()

    t0 = time.perf_counter()
    for i, point in enumerate(pos):
        pos = point
        G.add_node(i, x=pos[0], y=pos[1], z=pos[2])
    t2 = time.perf_counter()-t0

    t0 = time.perf_counter()
    for path in simplices3d.tolist():
        path.append(path[0])
        nx.add_path(G, path)
    t3 = time.perf_counter()-t0

    t0 = time.perf_counter()
    distances = {edge: distance(tolist(G, edge[0]), tolist(G, edge[1])) for edge in G.edges}
    nx.set_edge_attributes(G, values=distances, name='distance')
    t4 = time.perf_counter()-t0

    nx.write_graphml(G, graph_path)

    print(f'Tiempo total: {t1+t2+t3+t4} s')
